# 1. 필요한 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 2. 데이터 로드

In [2]:
# UCI-HAR 데이터 로드
def load_uci_har_data():
    uci_har_path = './원본 데이터/UCI HAR Dataset/'

    gyro_x_train = pd.read_csv(uci_har_path + 'train/Inertial Signals/body_gyro_x_train.txt', sep='\s+', header=None).values
    gyro_y_train = pd.read_csv(uci_har_path + 'train/Inertial Signals/body_gyro_y_train.txt', sep='\s+', header=None).values
    gyro_z_train = pd.read_csv(uci_har_path + 'train/Inertial Signals/body_gyro_z_train.txt', sep='\s+', header=None).values
    labels_train = pd.read_csv(uci_har_path + 'train/y_train.txt', sep='\s+', header=None).values - 1
    X_train = np.stack([gyro_x_train, gyro_y_train, gyro_z_train], axis=-1)
    return X_train, labels_train

X_train, y_train = load_uci_har_data()

# UCI-HAR 데이터 DataFrame으로 변환
uci_har_df = pd.DataFrame(X_train.reshape(-1, 3), columns=['X', 'Y', 'Z'])

In [3]:
# 자이로 데이터 로드
gyro_data = pd.read_csv('./원본 데이터/자이로 데이터.csv')

print("UCI-HAR 데이터 샘플:")
print(uci_har_df.head())
print("\n자이로 데이터 샘플:")
print(gyro_data.head())

UCI-HAR 데이터 샘플:
          X         Y         Z
0  0.030191  0.066014  0.022859
1  0.043711  0.042699  0.010316
2  0.035688  0.074850  0.013250
3  0.040402  0.057320  0.017751
4  0.047097  0.052343  0.002553

자이로 데이터 샘플:
   WorkDate  UserCode     X     Y     Z         RegisterDate
0  20230803         1 -0.01  0.00  0.00  2023-08-03 10:34:41
1  20230803         1  0.05 -0.02 -0.02  2023-08-03 10:32:47
2  20230803         1  0.06  0.12  0.08  2023-08-03 10:30:12
3  20230803         1  0.02  0.02 -0.01  2023-08-03 10:33:41
4  20230803         1  0.17  0.05  0.01  2023-08-03 10:33:03


# 3. 기본 통계 비교

In [4]:
def compare_basic_stats(uci_data, gyro_data):
    stats = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
    uci_stats = uci_data.describe().loc[stats]
    gyro_stats = gyro_data[['X', 'Y', 'Z']].describe().loc[stats]

    comparison = pd.concat([uci_stats, gyro_stats], axis=1, keys=['UCI-HAR', 'Gyro'])
    print("기본 통계 비교:")
    print(comparison)
    print("\n")

compare_basic_stats(uci_har_df, gyro_data)

기본 통계 비교:
       UCI-HAR                              Gyro                            
             X         Y         Z             X             Y             Z
mean  0.000506 -0.000824  0.000113  -3702.910910   2829.783692    966.719159
std   0.406815  0.381855  0.255743   2996.991394   4704.889694   3819.702620
min  -4.733656 -5.974330 -2.763014 -32768.000000 -30016.000000 -32768.000000
25%  -0.044328 -0.055065 -0.025253  -6080.000000   -256.000000  -1280.000000
50%   0.000138 -0.000578  0.000490  -3712.000000   3392.000000    576.000000
75%   0.054847  0.036958  0.049530  -1792.000000   7104.000000   3584.000000
max   4.155473  5.746062  2.365982  32512.000000  32384.000000  32640.000000




# 4. 분포 시각화

In [5]:
def plot_distributions(uci_data, gyro_data):
    fig, axes = plt.subplots(3, 1, figsize=(12, 15))
    for i, axis in enumerate(['X', 'Y', 'Z']):
        sns.kdeplot(uci_data[axis], ax=axes[i], label='UCI-HAR')
        sns.kdeplot(gyro_data[axis], ax=axes[i], label='Gyro')
        axes[i].set_title(f'{axis}-axis Distribution')
        axes[i].legend()
    plt.tight_layout()
    plt.savefig('./원본 데이터/UCI-HAR, 자이로 데이터 분포 시각화.png')
    plt.close()

plot_distributions(uci_har_df, gyro_data)
print("분포 비교 그래프가 'UCI-HAR, 자이로 데이터 분포 시각화.png'로 저장되었습니다.")

분포 비교 그래프가 'UCI-HAR, 자이로 데이터 분포 시각화.png'로 저장되었습니다.


# 5. 주파수 도메인 분석:

In [7]:
def compare_frequency_domain(uci_data, gyro_data):
    def fft_analysis(data):
        # 데이터를 numpy 배열로 변환
        data_array = np.array(data)
        # 데이터 길이를 동일하게 맞춤 (더 짧은 데이터에 맞춤)
        min_length = min(len(uci_data), len(gyro_data))
        data_array = data_array[:min_length]
        
        fft_result = np.fft.fft(data_array)
        freq = np.fft.fftfreq(len(data_array))
        return freq, np.abs(fft_result)
    
    fig, axes = plt.subplots(3, 2, figsize=(15, 15))
    for i, axis in enumerate(['X', 'Y', 'Z']):
        uci_freq, uci_amp = fft_analysis(uci_data[axis])
        gyro_freq, gyro_amp = fft_analysis(gyro_data[axis])
        
        axes[i, 0].plot(uci_freq, uci_amp, label='UCI-HAR')
        axes[i, 0].set_title(f'UCI-HAR {axis}-axis Frequency')
        axes[i, 1].plot(gyro_freq, gyro_amp, label='Gyro')
        axes[i, 1].set_title(f'Gyro {axis}-axis Frequency')
        
        axes[i, 0].legend()
        axes[i, 1].legend()
    
    plt.tight_layout()
    plt.savefig('frequency_domain_comparison.png')
    plt.close()

compare_frequency_domain(uci_har_df, gyro_data)
print("주파수 도메인 비교 그래프가 'frequency_domain_comparison.png'로 저장되었습니다.")

주파수 도메인 비교 그래프가 'frequency_domain_comparison.png'로 저장되었습니다.


# 6. 상관관계 분석:

In [8]:
def compare_correlation(uci_data, gyro_data):
    uci_corr = uci_data.corr()
    gyro_corr = gyro_data[['X', 'Y', 'Z']].corr()

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    sns.heatmap(uci_corr, annot=True, cmap='coolwarm', ax=ax1)
    ax1.set_title('UCI-HAR Correlation Matrix')
    sns.heatmap(gyro_corr, annot=True, cmap='coolwarm', ax=ax2)
    ax2.set_title('Gyro Correlation Matrix')
    plt.savefig('./원본 데이터/UCI-HAR, 자이로 데이터 상관관계 분석.png')
    plt.close()

compare_correlation(uci_har_df, gyro_data)
print("상관관계 분석 그래프가 'UCI-HAR, 자이로 데이터 상관관계 분석.png'로 저장되었습니다.")

상관관계 분석 그래프가 'UCI-HAR, 자이로 데이터 상관관계 분석.png'로 저장되었습니다.


# 7. 시계열 패턴 비교

In [9]:
def compare_time_series(uci_data, gyro_data, samples=1000):
    fig, axes = plt.subplots(3, 2, figsize=(15, 15))
    for i, axis in enumerate(['X', 'Y', 'Z']):
        axes[i, 0].plot(uci_data[axis][:samples])
        axes[i, 0].set_title(f'UCI-HAR {axis}-axis Time Series')
        axes[i, 1].plot(gyro_data[axis][:samples])
        axes[i, 1].set_title(f'Gyro {axis}-axis Time Series')

    plt.tight_layout()
    plt.savefig('./원본 데이터/UCI-HAR, 자이로 데이터 시계열 패턴 비교.png')
    plt.close()

compare_time_series(uci_har_df, gyro_data)
print("시계열 패턴 비교 그래프가 'UCI-HAR, 자이로 데이터 시계열 패턴 비교.png'로 저장되었습니다.")

시계열 패턴 비교 그래프가 'UCI-HAR, 자이로 데이터 시계열 패턴 비교.png'로 저장되었습니다.


# 8. 결과 해석 및 요약

In [12]:
def summarize_findings(uci_data, gyro_data):
    print("데이터셋 간 주요 차이점:")
    
    # 평균 비교
    for axis in ['X', 'Y', 'Z']:
        uci_mean = uci_data[axis].mean()
        gyro_mean = gyro_data[axis].mean()
        print(f"{axis}-axis 평균 차이: {uci_mean - gyro_mean:.4f}")
    
    # 표준편차 비교
    for axis in ['X', 'Y', 'Z']:
        uci_std = uci_data[axis].std()
        gyro_std = gyro_data[axis].std()
        print(f"{axis}-axis 표준편차 차이: {uci_std - gyro_std:.4f}")
    
    # 상관관계 차이
    uci_corr = uci_data[['X', 'Y', 'Z']].corr()
    gyro_corr = gyro_data[['X', 'Y', 'Z']].corr()
    print("\n상관관계 차이:")
    corr_diff = uci_corr - gyro_corr
    print(corr_diff)

summarize_findings(uci_har_df, gyro_data)

데이터셋 간 주요 차이점:
X-axis 평균 차이: 3702.9114
Y-axis 평균 차이: -2829.7845
Z-axis 평균 차이: -966.7190
X-axis 표준편차 차이: -2996.5846
Y-axis 표준편차 차이: -4704.5078
Z-axis 표준편차 차이: -3819.4469

상관관계 차이:
          X         Y         Z
X  0.000000 -0.236934 -0.285683
Y -0.236934  0.000000  0.266782
Z -0.285683  0.266782  0.000000


## 차이점
1. 데이터 구조:
> UCI-HAR 데이터는 X, Y, Z 축의 자이로스코프 데이터만 포함   
> 자이로 데이터는 추가적인 정보(WorkDate, UserCode, RegisterDate)를 포함   
2. 스케일 차이:
> UCI-HAR 데이터는 -6에서 6 사이의 값을 가짐   
> 자이로 데이터는 -32768에서 32640 사이의 값을 가짐   
> 이는 자이로 데이터가 raw 센서 데이터일 가능성이 높으며, 정규화되지 않았음을 시사   
3. 평균 차이:
> X, Y, Z 축 모두에서 큰 평균 차이가 있음
> 이는 두 데이터셋의 중심점이 매우 다르다는 것을 의미
4. 표준편차 차이:
> 자이로 데이터의 표준편차가 UCI-HAR 데이터보다 훨씬 큼
> 이는 자이로 데이터의 변동성이 더 크다는 것을 의미
5. 상관관계 차이:
> 축 간 상관관계에 차이가 있음
> 특히 X-Z축과 Y-Z축 간의 상관관계 차이가 크게 나타남

### 이러한 차이점을 고려하여 다음과 같은 조치를 취함

1. 데이터 정규화:
> 자이로 데이터를 UCI-HAR 데이터와 유사한 스케일로 정규화   
> 예를 들어, Min-Max 스케일링이나 표준화(Z-score)를 적용할 수 있습니다.
2. 중심 조정:
> 자이로 데이터의 평균을 0에 가깝게 조정   
> 이는 데이터의 중심을 맞추는 데 도움
3. 특성 공학:
> 자이로 데이터의 추가 정보(WorkDate, UserCode 등)를 활용하여 새로운 특성을 만들어낼 수 있음
4. 데이터 필터링:
> 자이로 데이터의 이상치를 제거하거나 필터링하여 UCI-HAR 데이터와 유사한 분포를 가지도록 조정 가능
5. 모델 조정:
> 두 데이터셋 간의 차이를 고려하여 모델의 아키텍처나 하이퍼파라미터를 조정할 필요가 있음   
> 이러한 조치를 통해 UCI-HAR 데이터로 학습한 모델을 자이로 데이터에 적용할 때 발생할 수 있는 문제를 최소화하고, 더 나은 성능을 얻을 수 있음